In [1]:
# Importing useful packages
import numpy as np
from scipy import stats
import pandas as pd
import sklearn as sk
import seaborn as sb
import datetime as dt
import pylab 
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from new_var import calc,y1function,y2function,C3function,C6function
from outliers import outlier
from Standardising import standard
from Recoding_SIC_Codes import Industry_Division 
%matplotlib inline
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.cross_validation import train_test_split
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.metrics import precision_recall_fscore_support


# Read in Data file and define NaN values
ipo_data = pd.read_excel("Competition #1 Raw Data_UPDATED I3.xlsx",header=0,na_values='-' )
ipo_data.I3 = ipo_data.I3.astype(object) # Converting to object for the moment to tidy up summary statistics
# Run Descriptive Statistics on Data File
#print(ipo_data.dtypes)
# Replacing NaN in C2 with 1 
ipo_data['C2']=ipo_data['C2'].fillna(1)


# Removing negative and 0 values for the word/sentence counts as it is beleived that these are errors
# Removing 0 and negatives for T' calculations        
ipo_data['T2']=ipo_data['T2'].mask(ipo_data['T2'] <= 0,ipo_data['T2'].mean())
ipo_data['T1']=ipo_data['T1'].mask(ipo_data['T1'] <= 0,ipo_data['T1'].mean())
ipo_data['T3']=ipo_data['T3'].mask(ipo_data['T3'] <= 0,ipo_data['T3'].mean())
ipo_data['T4']=ipo_data['T4'].mask(ipo_data['T4'] <= 0,ipo_data['T4'].mean())
ipo_data['T5']=ipo_data['T5'].mask(ipo_data['T5'] <= 0,ipo_data['T5'].mean())
ipo_data['S1']=ipo_data['S1'].mask(ipo_data['S1'] <= 0,ipo_data['S1'].mean())
ipo_data['C3']=ipo_data['C3'].mask(ipo_data['C3'] == 0,ipo_data['C3'].mean())
# Replace NaN values with mean
ipo_data=ipo_data.fillna(ipo_data.mean())


C:\Users\conor\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
C:\Users\conor\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:

# Creating our target and control variables
ipo_data['Y1'] = ipo_data.apply(y1function, axis=1)
ipo_data['Y2'] = ipo_data.apply(y2function, axis=1)
ipo_data['C3x'] = ipo_data.apply(C3function, axis=1)
ipo_data['C6x'] = ipo_data.apply(C6function, axis=1)

#Creating ratios for the words, sentences and postive/negative counts. Ratios provide insight
calc(ipo_data)

#Removing redundent data
# del ipo_data['C3']
# del ipo_data['C5']
# del ipo_data['C6']
# # del ipo_data['T1']
# # del ipo_data['T2']
# del ipo_data['T3']
# del ipo_data['T4']
# del ipo_data['T5']
# del ipo_data['S1']
# del ipo_data['S2']
# del ipo_data['S3']
# del ipo_data['P(1Day)']
# del ipo_data['P(L)']
# del ipo_data['P(H)']
# del ipo_data['P(IPO)']
#Looking at the new description of the data
ipo_data.describe()

,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,C4,C5,C6,...,Y2,C3x,C6x,C5x,T3x,T4x,T5x,S1x,S2x,S3x
count,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,6.820000e+02,6.820000e+02,...,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000
mean,13.837666,15.481190,13.515045,25.934766,149.728788,0.863636,1.802750,0.007282,4.935776e+07,1.241519e+07,...,0.708211,0.536657,0.052708,4.627711,0.918942,0.629876,0.080692,0.005504,0.009179,0.011854
std,6.031467,6.604398,5.792641,72.042293,150.328785,0.343426,158.308525,0.032775,1.039156e+08,2.501761e+07,...,0.454919,0.499020,0.107639,6.245584,0.624282,0.076265,0.744766,0.004784,0.004324,0.008970
min,3.000000,0.000000,3.000000,0.000000,10.000000,0.000000,-786.239000,-0.162352,3.693227e+06,5.250000e+05,...,0.000000,0.000000,0.000000,0.283223,0.703925,0.395480,0.031595,0.001389,0.002152,0.005125
25%,10.000000,13.000000,11.000000,11.092500,85.000000,1.000000,-0.750000,-0.013352,1.874682e+07,5.004180e+06,...,0.000000,0.000000,0.000000,2.853439,0.883578,0.580815,0.047413,0.004154,0.007031,0.009570
50%,13.500000,15.481190,13.000000,15.000000,110.000000,1.000000,0.085000,0.007282,2.759165e+07,7.500000e+06,...,1.000000,1.000000,0.000000,3.770051,0.897133,0.629077,0.051901,0.005069,0.008600,0.011305
75%,17.000000,17.000000,15.000000,21.030001,152.750000,1.000000,0.620000,0.030941,4.938184e+07,1.208721e+07,...,1.000000,1.000000,0.071429,4.865080,0.909732,0.674852,0.056471,0.006258,0.010535,0.012883
max,85.000000,135.000000,108.000000,1159.200562,2087.000000,1.000000,3864.500000,0.092896,2.138085e+09,4.212336e+08,...,1.000000,1.000000,1.000000,99.787255,17.165085,1.443089,19.500949,0.121442,0.091082,0.233397


In [3]:
ipo_data['income']=ipo_data['P(IPO)']*ipo_data['C6']
ipo_data['perOffered']=ipo_data['C6']/ipo_data['C5']
ipo_data['P/E']=ipo_data['P(IPO)']/ipo_data['C3']
ipo_data['P/E']=ipo_data['P/E']+1150
ipo_data.describe()

,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,C4,C5,C6,...,C5x,T3x,T4x,T5x,S1x,S2x,S3x,income,perOffered,P/E
count,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,6.820000e+02,6.820000e+02,...,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,6.820000e+02,682.000000,682.000000
mean,13.837666,15.481190,13.515045,25.934766,149.728788,0.863636,1.802750,0.007282,4.935776e+07,1.241519e+07,...,4.627711,0.918942,0.629876,0.080692,0.005504,0.009179,0.011854,2.265801e+08,0.304365,1159.904779
std,6.031467,6.604398,5.792641,72.042293,150.328785,0.343426,158.308525,0.032775,1.039156e+08,2.501761e+07,...,6.245584,0.624282,0.076265,0.744766,0.004784,0.004324,0.008970,9.485423e+08,0.221202,157.354155
min,3.000000,0.000000,3.000000,0.000000,10.000000,0.000000,-786.239000,-0.162352,3.693227e+06,5.250000e+05,...,0.283223,0.703925,0.395480,0.031595,0.001389,0.002152,0.005125,1.706250e+06,0.010021,0.000000
25%,10.000000,13.000000,11.000000,11.092500,85.000000,1.000000,-0.750000,-0.013352,1.874682e+07,5.004180e+06,...,2.853439,0.883578,0.580815,0.047413,0.004154,0.007031,0.009570,5.810000e+07,0.205547,1139.437996
50%,13.500000,15.481190,13.000000,15.000000,110.000000,1.000000,0.085000,0.007282,2.759165e+07,7.500000e+06,...,3.770051,0.897133,0.629077,0.051901,0.005069,0.008600,0.011305,9.775000e+07,0.265248,1152.913835
75%,17.000000,17.000000,15.000000,21.030001,152.750000,1.000000,0.620000,0.030941,4.938184e+07,1.208721e+07,...,4.865080,0.909732,0.674852,0.056471,0.006258,0.010535,0.012883,1.734750e+08,0.350455,1177.990302
max,85.000000,135.000000,108.000000,1159.200562,2087.000000,1.000000,3864.500000,0.092896,2.138085e+09,4.212336e+08,...,99.787255,17.165085,1.443089,19.500949,0.121442,0.091082,0.233397,1.786400e+10,3.530786,2950.000000


In [4]:
# Converting C6x and C2 to correct type
ipo_data.C6x = ipo_data.C6x.astype(float)

ipo_data.C2 = ipo_data.C2.astype(int)
#Filing days was broken into three seperate groups. Under six month, 6-12 months, and over 12 months. 
ipo_data['C6xbin'] = 0
ipo_data.loc[(ipo_data['C6x'] > 0),'C6xbin'] = 1

sqrt_transform=['income','P/E','perOffered','S1x','S2x','S3x'] #Variables that need to be square rooted
ipo_data[sqrt_transform]=ipo_data[sqrt_transform]**0.5 #square rooting variable

log_transform=['C1','C7'] #Variables to be log transformed
ipo_data[log_transform]=np.log(ipo_data[log_transform])#log transformation

ipo_data['T5xbin'] = 0
# ipo_data.loc[(ipo_data['T5x'] >= 0.080692),'T5xbin'] = 1
ipo_data.loc[(ipo_data['T5x'] >= 0) & (ipo_data['T5x'] < .0474),'T5xbin'] = 1
ipo_data.loc[(ipo_data['T5x'] >= .047401) & (ipo_data['T5x'] < .0519), 'T5xbin'] = 2
ipo_data.loc[(ipo_data['T5x'] >= .051901) & (ipo_data['T5x'] < .05647), 'T5xbin'] = 3
ipo_data.loc[ipo_data[ 'T5x'] >= .0564701,'T5xbin'] = 4

ipo_data['T3xbin'] = 0
# ipo_data.loc[(ipo_data['T3x'] < 0.897133),'T3xbin'] = 0
# ipo_data.loc[(ipo_data['T3x'] >= 0.897133),'T3xbin'] = 1
ipo_data.loc[(ipo_data['T3x'] >= 0.703925) & (ipo_data['T3x'] < 0.883578),'T3xbin'] = 1
ipo_data.loc[(ipo_data['T3x'] >= .883578) & (ipo_data['T3x'] < 0.897133), 'T3xbin'] = 2
ipo_data.loc[(ipo_data['T3x'] >= .897133) & (ipo_data['T3x'] < 0.909732), 'T3xbin'] = 3
ipo_data.loc[ipo_data[ 'T3x'] >= 0.909732,'T3xbin'] = 4

print(ipo_data.dtypes) # checking the types
print(ipo_data.T5xbin.value_counts())

I1             object
I2             object
I3             object
P(IPO)        float64
P(H)          float64
P(L)          float64
P(1Day)       float64
C1            float64
C2              int32
C3            float64
C4            float64
C5            float64
C6            float64
C7            float64
T1            float64
T2            float64
T3            float64
T4            float64
T5            float64
S1            float64
S2            float64
S3            float64
Y1              int64
Y2              int64
C3x             int64
C6x           float64
C5x           float64
T3x           float64
T4x           float64
T5x           float64
S1x           float64
S2x           float64
S3x           float64
income        float64
perOffered    float64
P/E           float64
C6xbin          int64
T5xbin          int64
T3xbin          int64
dtype: object
4    171
2    171
3    170
1    170
Name: T5xbin, dtype: int64


In [6]:
"""Code to recode the column I3 into industry level.

    Version Control:
    Initial coding
    ------------------------------------------
    Date 8-Feb-18, Author: Danielle Ezzo, Desc: Initial Coding
    """

def Industry_Division1(row,method):
    # Recoding based on table in milestone report
    if method==1:
        if (row['I3']> 99 and row['I3']< 1000):
            return 'Agriculture, Forestry and Fishing'
        elif (row['I3']> 999 and row['I3']< 1500):
            return  'Mining'
        elif (row['I3']> 1499 and row['I3']< 1800):
            return 'Construction'
        elif (row['I3']> 1999 and row['I3']< 4000):
            return 'Manufacturing'
        elif (row['I3']> 3999 and row['I3']< 5000):
            return 'Transportation, Communications, Electric, Gas and Sanitary service'
        elif (row['I3']> 4999 and row['I3']< 5200):
            return 'Wholesale Trade'
        elif (row['I3']> 5199 and row['I3']< 6000):
            return 'Retail Trade'
        elif (row['I3']> 5999 and row['I3']< 6800):
            return 'Finance, Insurance and Real Estate'
        elif (row['I3']> 6999 and row['I3']< 9000):
            return 'Services'
        elif (row['I3']> 9099 and row['I3']< 9730):
            return 'Public Administration'
        elif (row['I3']> 9899 and row['I3']< 10000):
            return 'Nonclassifiable'
        else:
            return 'Missing'
    elif method==2:
        if (row['I3']> 1999 and row['I3']< 4000):
            return 'Manufacturing'
        else:
            return 'Other'
    elif method==3:
        if (row['I3']> 7369 and row['I3']< 7375) :
            return 'Tech'
        else:
            return 'NonTech'
    elif method==4:
        if (row['I3']> 1999 and row['I3']< 4000):
            return 'Manufacturing' # Manufacturing
        elif (row['I3']> 3999 and row['I3']< 5000):
            return 'Services' # Tech Comms etc Services
        elif (row['I3']> 6999 and row['I3']< 9000):
            return 'Services' #Services (combining with above)
        else:
            return 'Other' # All Other
    elif method==5:
        if(row['I3']> 1999 and row['I3']< 4000):
            return 'Manufacturing'
        elif (row['I3']> 3999 and row['I3']< 5000):
            return 'Services' #TCEGS
        elif (row['I3']> 4999 and row['I3']< 5200):
            return 'Trade' #wholesale
        elif (row['I3']> 5199 and row['I3']< 6000):
            return 'Trade' #retail
        elif (row['I3']> 6999 and row['I3']< 9000):
            return 'Services'
        else:
            return 'Other'
    elif method==6:
        if(row['I3']> 1999 and row['I3']< 4000):
            return 'Manufacturing'
        elif (row['I3']> 3999 and row['I3']< 5000):
            return 'TCEGS' 
        elif (row['I3']> 4999 and row['I3']< 5200):
            return 'Trade' #wholesale
        elif (row['I3']> 5199 and row['I3']< 6000):
            return 'Trade' #retail
        elif (row['I3']> 6999 and row['I3']< 9000):
            return 'Services'
        else:
            return 'Other'
    elif method==7:
        if (row['I3']> 99 and row['I3']< 5000):
            return 'Manual Labor'
        else: 
            return 'NonManual Labor'

# Add the new data to the end of the table
ipo_data.I3 = ipo_data.I3.astype(int) # recoding to correct type

# import functools
# import operator


# applying our function to recode our industry codes to sector level
ipo_data['IndDivision'] = ipo_data.apply(Industry_Division1,method=4, axis=1) 
print(ipo_data.head(5))

     I1                           I2    I3  P(IPO)  P(H)  P(L)    P(1Day)  \
0    RA         RAILAMERICA INC /DE   4011    15.0  18.0  16.0  13.750000   
1     G                 Genpact LTD   8742    14.0  18.0  16.0  16.750000   
2  HURN  Huron Consulting Group Inc.  8742    15.0  16.0  14.0  18.900000   
3  ICFI     ICF International, Inc.   8742    12.0  16.0  14.0  12.250000   
4  SRVY       GREENFIELD ONLINE INC   8742    13.0  15.0  13.0  18.700001   

         C1  C2    C3     ...            S1x       S2x       S3x  \
0  4.343805   1  0.30     ...       0.082570  0.122955  0.081668   
1  4.418841   1  0.19     ...       0.077702  0.086114  0.109887   
2  5.030438   1 -0.14     ...       0.055644  0.070385  0.090297   
3  4.941642   1  0.22     ...       0.075398  0.081162  0.126022   
4  4.553877   1  0.07     ...       0.081974  0.095998  0.109543   

         income  perOffered        P/E  C6xbin  T5xbin  T3xbin  IndDivision  
0  18165.902125    0.636244  34.641016       0    

In [7]:
dummies = pd.get_dummies(ipo_data['IndDivision'])

ipo_data = ipo_data.join(dummies)

In [8]:

x=pd.DataFrame.copy(ipo_data)
for i in range(len(x.columns)): 
    if x.iloc[:,i].dtype == float:
        outlier(x.iloc[:,i],4)             
for i in range(len(x.columns)):        
    if x.iloc[:,i].dtype == float:   
        x.iloc[:,i]=standard(x.iloc[:,i],1)


# Create K-means model and determine euclidian distances for each data point
#train=['C3x','C1','T3xbin','T4x','T5xbin','S1x','S2x','S3x','C2','T2','T1','C6x','Other']
train=['perOffered','income','P/E','C3x','C1','T3xbin','T4x','S1x','S2x','S3x','C2','T2','T1','C6xbin','Other','Manufacturing']
X=x[train]
kmeans_model = KMeans(n_clusters=2)
distances = kmeans_model.fit_transform(X)

# Create scatter plot using labels from K-means model as color
labels = kmeans_model.labels_


x['labels'] = labels
train.append('labels')

C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = lq
C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = uq


In [9]:
print(x.head(5))

     I1                           I2    I3    P(IPO)    P(H)    P(L)  \
0    RA         RAILAMERICA INC /DE   4011  0.500000  0.6875  0.6875   
1     G                 Genpact LTD   8742  0.458333  0.6875  0.6875   
2  HURN  Huron Consulting Group Inc.  8742  0.500000  0.5625  0.5625   
3  ICFI     ICF International, Inc.   8742  0.375000  0.5625  0.5625   
4  SRVY       GREENFIELD ONLINE INC   8742  0.416667  0.5000  0.5000   

    P(1Day)        C1  C2        C3   ...    perOffered       P/E  C6xbin  \
0  0.382795  0.327835   1  0.566929   ...      0.713752  0.768255       0   
1  0.466314  0.360190   1  0.545276   ...      0.290540  0.919586       0   
2  0.526169  0.623910   1  0.480315   ...      0.557648  0.374358       0   
3  0.341036  0.585622   1  0.551181   ...      0.646033  0.797414       0   
4  0.520601  0.418417   1  0.521654   ...      0.551954  0.626279       0   

   T5xbin  T3xbin  IndDivision  Manufacturing  Other  Services  labels  
0       1       1     Services 

In [10]:
x.to_csv('Model 28_02_18.csv')